# Labrapport som Jupyter Notebook på fysikklab våren 2023
Skrevet 16.03.2023 <br>
Skrevet av Emma Amalie Chun Hui Birkhaug Pedersen, Halvor Linder Henriksen, Oskar Jørgensen og Sindre Eidskrem <br>
Institutt for Fysikk, NTNU
***

## Sammendrag
Sammendraget inneholder en konsis oppsummering av innholdet i oppgaven. Det kan være en god idé å skrive denne delen til slutt, når du har god oversikt over hele teksten. Typisk inneholder sammendraget 4-5 setninger. På disse få setningene skal førsøket introduseres, de eksperimentelle og numeriske metodene nevnes, og resultatene - og deres betydning - skal presenteres. Sentrale tallverdier oppgis med tilhørende usikkerhet.

***

## Introduksjon
Selve rapporten starter her. Introduksjonen bør inneholde en tydelig motivasjon og hensikt ved forsøket. Her kan også teksten settes i en historisk sammenheng. Det vil i så fall være natulig å referere til relevant litteratur.
***

## Teori
Her presenteres relevant teori nødvendig for å lese resten av rapporten. Jupyter notebook støtter LaTeX-kode, så dere kan inkludere fint formaterte ligninger her. For eksempel har vi sett at baneakselerasjonen til ei kompakt kule som ruller på et krumt underlag uten å slure (gli) er gitt ved
$$a = -\frac{5g\sin{\beta}}{7},\label{eq1}\tag{1}$$
der $g$ er tyngdens akselerasjon og $\beta$ er banens helningsvinkel. Her forklarer vi alle nye størrelser som blir innført med ord. <br>
For at dette skal ha god flyt, skal ligningene kunne leses som en naturlig del av setningen, slik som ovenfor. 

Teoridelen inneholder ikke referanser til selve forsøket. Den er generell og kan leses isolert fra resten av rapporten.

Her vil det ofte være naturlig å inkludere en eller flere forklarende figurer. Disse skal dere lage selv, for eksempel ved hjelp av [Inkscape](https://inkscape.org/) som er et nokså brukervennlig program for å lage vektorgrafikk. Her er et eksempel på en slik figur:

In [1]:
from IPython import display
display.Image("./krefter.png", width=300)

FileNotFoundError: No such file or directory: './krefter.png'

FileNotFoundError: No such file or directory: './krefter.png'

<IPython.core.display.Image object>

<b>Figur 1.</b> Kule som ruller på et krumt underlag. Her er $v$ kulas fart,
$a$ er baneakselerasjon, $a_\perp$ er sentripetalakselerasjon, $Mg$ er tyngdekraft,
$f$ er friksjonskraft og $N$ er normalkraft. <br><br>

***

## Metode

### Utstyr

- Gummikule med vekt 30.5g og radius 19.8cm
- Kamera som filmer med 30fps
- Tripod
- Bane på 1.4m horisontal lengde med 8 justerbare festepunkter
- Vekt
- Linjal
- Tracker (Software)

### Beregninger
Baneformen bestemmes ved å generere 7 tilfeldige høydeverdier i intervallet 0-300mm, for så å tilpasse tredjegradspolynomer mellom punktene. Avstanden mellom hvert festepunk er 200mm i x-retning. 
Dette gjøres i Python med funksjonen CubicSpline fra SciPy-biblioteket interpolate
med kommandoen

```python
cs = CubicSpline(xfast,yfast,bc_type='natural')
```

Her er xfast og yfast tabeller (numpy arrays) med henholdsvis horisontale og vertikale koordinater 
for skruefestene, og med *bc_type='natural'* velges krumning lik null i de to ytterste skruefestene.

Fra den resulterende funksjonen vil da være 2 ganger kontunierlig deriverbar, og krumningen vil også være kontinuerlig. Funksjonens første- og andrederiverte, så vel som krumningen kan så regnes ut ved hjelp av funksjonen cs:

```python
y = cs(x)
```  
```python
dy = cs(x,1)
```  
```python
d2y = cs(x,2)
```  

```python
curvature = d2y / np.power((1 + np.power(dy, 2)), 3/2)
```

Videre kan banens helningsvinkel regnes ut ved:  
```python
beta = [np.arctan(x) for x in dy]
```

Her er x et array som representerer heltallige x-verdier i intervallet 0-1400mm. Det andre argumentet til cs er antall ganger funksjonen skal deriveres.

Kulens hastighet som funksjon av x kan regnes ut ved hjelp av energibevaring på følgende måte:  
```python
vel = [np.sqrt(10/7 * g * (y[0] - y[x])) for x in range(len(y))]
```

Hastigheten kan så dekomponeres for å finne hastigheten i x-retning:  
```python
x_vel = [v * np.cos(b) for v, b in zip(vel, beta)]
```

Da er gjennomsnittlig x-hastighet på hvert millimeter-intervall gitt ved:  
```python
x_avg_vel = [(x_vel[i-1] + x_vel[i])/2 for i in range(1, len(x_vel))]
```

Tiden brukt på hvert x-intervall kan så finnes ved:  
```python
delta_t = [1/x for x in x_avg_vel]
```

Den totale tiden frem til hvert x-punkt kan deretter regnes ut med en cumulativ sum:  
```python
time = np.cumsum(delta_t)
```
Starttiden t(0)=0 kan deretter legger til manuelt:
```python
time = np.insert(time, 0, 0, axis=0)
```

Sentripetalakselerasjonen gitt x kan så regnes ut fra kulens hastighet og banens krumning i x:  
```python
sentripetal_acceleration = np.multiply(np.power(np.array(vel), 2), curvature)
```

Videre kan normalkraften beregnes fra banens helningsvinkel og kulens sentripetalakselerasjon:  
```python
M = 0.0305
c = 2/5

normal_force = M * (g * np.cos(np.array(beta)) + sentripetal_acceleration)
```
Her er M kulens masse målt i kg, mens c er c-verdien for en kule.

Kulens baneakselerasjon kan så regnes ut fra banens helningsvinkel:  
```python
path_acceleration = - (5*g*np.sin(np.array(beta))/7)
```

Videre kan friksjonskraften gitt x også også regnes ut ved helningsvinkelen:
```python
friction = c / (1+c) * M*g*np.sin(np.array(beta))
```



### Målinger

Den fysiske banens festepunkter fastsettes til de genererte høydene, med nullnivå ved banestativets base. 
Deretter festes kameraet til tripoden slik at planet definert av kameralinsen er parallelt med banens baktavle, samt at hele banen synes på filmen. 

Videre filmes 10 forsøk der ballen plasseres på banen over det første festepunktet og slippes uten å støtes langs banen.
Forsøkensvideoene analyseres så i Tracker, før data for kulens x- og y-koordinater gitt ved tid relativt til starttidspunktet eksporteres til CSV.

Deretter analyseres målingene i Python og sluttiden kan regnes ut som den numeriske tidsderiverte av den målte forflytningen mellom de to siste kamerabildene.  

```python
def get_end_velocity(file_path):
    time_before, x_before, y_before, time_after, x_after, y_after = get_values_from_file(
        file_path)
    return math.sqrt((x_after-x_before)**2+(y_after-y_before)**2) / (time_after - time_before)


for i in range(1, 11):
    end_vel = get_end_velocity(f'kule_{ i }_data.txt')
```
Her er *get_values_from_file* en funksjon som returner målingene for de to siste målepunktene.

Kulens totale kinetiske energi kan så regnes ut gitt dens slutthastighet:  
```python
Ks = np.zeros(10)
for i in range(1,11):
    K = (1+c)/2 * M * (get_end_velocity(f"kule_{i}_data.txt")/1000)**2
    Ks[i-1] = K
```
Her deles hastigheten på 1000 for å konvertere fra mm/s til m/s

Endringen i total energi regnes så ut med tilsvarende skalering:  
```python
DEs = np.zeros(10)
for i in range(1,11):
    _, _, _, _, _, y_after = get_values_from_file(f"kule_{i}_data.txt")
    delta_e = m*(g/1000*(300-y_after)/1000-0.5*(get_end_velocity(f"kule_{i}_data.txt")/1000)**2)
    DEs[i-1] = delta_e
```

Til slutt kan gjennomsnitt, standardavvik og standardfeil regnes ut med innebygde numpy-funksjoner:  
```python
    np.mean(DEs)
    np.std(DEs)
    np.std(DEs)/np.sqrt(10)
```


***

## Resultater
Her presenteres numeriske og eksperimentelle resultater. Dere inkluderer figurer i henhold til rapportkravene, samt tallverdier for kulas rulletid, slutthastighet, tilhørende kinetisk energi, og tap i mekanisk energi, med tilhørende usikkerhet. Anvendt pythonkode inngår naturlig her. En må forvente at dette avsnittet opptar størst plass i rapporten. 
***

#### Baneform
Her er den teoretiske baneformen, samt vår målte baneform. Vi skalerte våre målte verdier slik at differansen mellom første og siste x- og y-måling var lik.

![Målt baneform](measured_baneform.png)

#### Fart
Videre regnet vi ut både teoretisk og målt fart. Merk at vi igjen brukte de skalerte x- og y-verdiene til å regne ut kulas fart.

![Målt fart](velocity_measured.png)

#### Beregnet friksjonskraft og normalkraft 
Her er beregnet friksjonskraft $f(x)$ og normalkraft $N(x)$.
![Friksjonskraft og normalkraft](normal_force_friction.png)

Her er absoluttverdien til forholdet mellom de to kreftene plottet, $abs(f(x) / N(x))$
![Forhold, krefter](ratio_forces.png)


#### Målt rulletid, slutthastighet, tilhørende kinetisk energi og tap i mekanisk energi 

**Rulletid**

Den målte rulletiden er som følger:
 - Forsøk 1:  1.533 s
 - Forsøk 2:  1.534 s
 - Forsøk 3:  1.6   s
 - Forsøk 4:  1.567 s
 - Forsøk 5:  1.6   s
 - Forsøk 6:  1.567 s
 - Forsøk 7:  1.6   s
 - Forsøk 8:  1.5   s
 - Forsøk 9:  1.533 s
 - Forsøk 10: 1.5   s

 Dette gir gjennomsittlig rulletid på 1.553 s med en standardfeil på 0.0117 s


**Slutthastighet**

Den målte slutthastigheten er som følger:
 - Forsøk 1:  1071.385 mm/s
 - Forsøk 3:  1200.239 mm/s
 - Forsøk 2:  1135.609 mm/s
 - Forsøk 4:  1055.543 mm/s
 - Forsøk 5:  996.322  mm/s
 - Forsøk 6:  1145.994 mm/s
 - Forsøk 7:  1005.951 mm/s
 - Forsøk 8:  1227.327 mm/s
 - Forsøk 9:  1088.243 mm/s
 - Forsøk 10: 1195.584 mm/s

Dette gir gjennomsittlig sluttfart på 1112.220 mm/s med en standardfeil på 24.457 mm/s

**Kinetisk energi**

Den beregnede kinetiske energien ved slutten er som følger:

 - Forsøk 1: 0.0245 J
 - Forsøk 2: 0.0275 J
 - Forsøk 3: 0.0307 J
 - Forsøk 4: 0.0237 J
 - Forsøk 5: 0.0211 J
 - Forsøk 6: 0.0280 J
 - Forsøk 7: 0.0216 J
 - Forsøk 8: 0.0321 J
 - Forsøk 9: 0.0252 J
 - Forsøk 10: 0.0305 J

 Dette gir en gjennomsnittlig kinetisk energi på 0.0265 J med en standardfeil på 0.00116 J.

**Tap i mekanisk energi**

Det beregnede tapet i mekanisk energi er som følger:

 - Forsøk 1:  0.0396 J
 - Forsøk 2:  0.0374 J
 - Forsøk 3:  0.0351 J
 - Forsøk 4:  0.0401 J
 - Forsøk 5:  0.0420 J
 - Forsøk 6:  0.0371 J
 - Forsøk 7:  0.0417 J
 - Forsøk 8:  0.0341 J
 - Forsøk 9:  0.0390 J
 - Forsøk 10: 0.0353 J

 Dette gir et gjennomsnittlig tap på 0.0382 J med en standardfeil på 0.000829 J.

## Diskusjon
Her diskuteres resultatene, både de numeriske og de eksperimentelle. <br>

- Kan forskjellene mellom numerisk og eksperimentell hastighet $v(x)$ forklares med utgangspunkt i
forskjellene mellom numerisk og eksperimentell baneform $y(x)$?
- Er grafene for $v(x)$, $N(x)$ og $f(x)$ rimelige med utgangspunkt i baneformen $y(x)$? 
Vurder spesielt topp-punkt, bunnpunkt(er) og vendepunkter.
- Er det sannsynlig at baneformen gir ren rulling? Dette avhenger av at beregnet friksjonskraft hele veien er mindre enn maksimal statisk friksjonskraft, som er gitt ved $\mu_s N$. Grafen for $|f(x)/N(x)|$ gir grunnlag for å diskutere dette.
- Er målt rulletid rimelig, med utgangspunkt i beregnet rulletid?
- Er målt tap i mekanisk energi rimelig? Diskuter feilkilder.
***

## Konklusjon
Dette er en kort oppsummering av det som ble gjort og hvilke resultater som ble oppnådd. Tallverdier for numerisk vs eksperimentell rulletid og sluttfart oppgis, samt tap i mekanisk energi med usikkerhet. 

## Referanser
En labrapport som denne vil typisk inneholde et *lite* antall referanser. Av den grunn
er det greit å nummerere disse manuelt. Hvis du sjekker ulike
tidsskrifter, vil du se at ulike konvensjoner for formatering av referanser benyttes.
Det viktige er at referanselista formateres systematisk. Nedenfor viser vi eksempler på
henvisning til tidsskriftartikler, bok og url, med formatering
som benyttet i tidsskriftet Physical Review Letters: <br>

[1] D. Xiao, W. Yao, and Q. Niu, Valley-Contrasting Physics in Graphene: Magnetic Moment and Topological Transport, Phys. Rev. Lett. <b>99</b>, 236809 (2007).  <br>

[2] D. Xiao, M.-C. Chang, and Q. Niu, Berry phase effects on electronic properties, 
Rev. Mod. Phys. <b>82</b>, 1959 (2010).  <br>

[3] T. Ihn, <i>Semiconductor Nanostructures: Quantum States and
Electronic Transport</i> (Oxford University Press, New York,
2010). <br>

[4] See Supplemental Materials at
<a href="http://link.aps.org/supplemental/10.1103/PhysRevLett.128.067702">
link.aps.org/supplemental/10.1103/PhysRevLett.128.067702</a>.
<br><br>

Til sammenligning, et par referanser fra en artikkel i Chemical Reviews: <br>

(1) Ayyagari, A. V.; Gwalani, B.; Muskeri, S.; Mukherjee, S.;
Banerjee, R. Surface degradation mechanisms in precipitationhardened high-entropy alloys. *npj Materials Degradation* <b>2018</b>, *2*, 33.  <br>
(2) Frankel, G. S.; Vienna, J. D.; Lian, J.; Scully, J. R.; Gin, S.; Ryan,
J. V.; Wang, J.; Kim, S. H.; Windl, W.; Du, J. A comparative review of
the aqueous corrosion of glasses, crystalline ceramics, and metals. *npj
Materials Degradation* <b>2018</b>, *2*, 15. <br>

Små, men *mange* forskjeller: type parentes i referansenummeret; plassering, parentesbruk og 
font for årstall; plassering av forbokstav i fornavn; *and* eller ikke *and* foran sistnevnte 
av tre eller flere forfattere. Ubetydelige valg i seg selv, men viktig at alt gjøres *systematisk*.
